In [1]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')
omopcursor = conn.cursor()

In [2]:
#!pip install "dask[distributed]" --upgrade

In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [4]:
df_OBSERVATION_PERIOD = pd.read_sql('select * from `OBSERVATION_PERIOD`', con=conn)

/tmp/ipykernel_410915/2976340141.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_OBSERVATION_PERIOD = pd.read_sql('select * from `OBSERVATION_PERIOD`', con=conn)


In [5]:
print('Length - OBSERVATION_PERIOD: ', len(df_OBSERVATION_PERIOD))

Length - OBSERVATION_PERIOD:  0


In [6]:
print(df_OBSERVATION_PERIOD.head())

Empty DataFrame
Columns: [observation_period_id, person_id, observation_period_start_date, observation_period_end_date, period_type_concept_id]
Index: []


In [7]:
import MySQLdb
import pandas as pd

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

mimicivconn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='mimiciv')
mimicivcursor = conn.cursor()

In [8]:
df_admissions_mimiciv = pd.read_sql('select subject_id, hadm_id, admittime, dischtime from admissions', con=mimicivconn)

/tmp/ipykernel_410915/917330468.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_admissions_mimiciv = pd.read_sql('select subject_id, hadm_id, admittime, dischtime from admissions', con=mimicivconn)


In [9]:
ddf_admissions_mimiciv = dd.from_pandas(df_admissions_mimiciv).compute()

In [10]:
df_admissions_mimiciv = ''

In [11]:
print(ddf_admissions_mimiciv)

        subject_id   hadm_id           admittime           dischtime
0         10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00
1         16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00
2         19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00
3         18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00
4         11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00
...            ...       ...                 ...                 ...
431226    10382924  29999723 2170-08-09 18:56:00 2170-08-10 18:30:00
431227    11326722  29999745 2160-07-05 22:24:00 2160-07-12 12:50:00
431228    12133002  29999809 2172-09-24 21:27:00 2172-09-30 17:04:00
431229    16572164  29999828 2180-10-27 07:15:00 2180-10-30 17:25:00
431230    16549335  29999928 2165-07-28 17:22:00 2165-07-31 11:53:00

[431231 rows x 4 columns]


In [12]:
df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group  from patients', con=mimicivconn)

/tmp/ipykernel_410915/850515151.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group  from patients', con=mimicivconn)


In [13]:
ddf_patients_mimiciv = dd.from_pandas(df_patients_mimiciv).compute()

In [14]:
df_patients_mimiciv = ''

In [2]:
print(ddf_patients_mimiciv.head())

NameError: name 'ddf_patients_mimiciv' is not defined

In [16]:
ddf_adm_pat_merged = ddf_admissions_mimiciv.merge(ddf_patients_mimiciv, how='inner', on='subject_id')
print(ddf_adm_pat_merged.head()) 

   subject_id   hadm_id           admittime           dischtime  anchor_year  \
0    10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00         2159   
1    16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00         2143   
2    19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00         2172   
3    18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00         2142   
4    11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00         2188   

  anchor_year_group  
0       2011 - 2013  
1       2008 - 2010  
2       2017 - 2019  
3       2011 - 2013  
4       2011 - 2013  


In [17]:
ddf_adm_pat_merged['median_anchor_year'] = ddf_adm_pat_merged['anchor_year_group'].str.split('-').apply(lambda x: int((int(x[0]) + int(x[1])) / 2))
ddf_adm_pat_merged['admittime_year'] = ddf_adm_pat_merged['admittime'].dt.year
ddf_adm_pat_merged['diff_year'] = ddf_adm_pat_merged['anchor_year'] - ddf_adm_pat_merged['median_anchor_year']
ddf_adm_pat_merged['year_of_admit'] = ddf_adm_pat_merged['admittime_year'] - ddf_adm_pat_merged['diff_year']
ddf_adm_pat_merged['month_of_admit'] = ddf_adm_pat_merged['admittime'].dt.month
ddf_adm_pat_merged['day_of_admit'] = ddf_adm_pat_merged['admittime'].dt.day
ddf_adm_pat_merged['day_of_admit'] = ddf_adm_pat_merged.apply(
    lambda row: min(row['day_of_admit'], pd.Period(year=row['year_of_admit'], month=row['month_of_admit'], freq='M').days_in_month),
    axis=1
)

ddf_adm_pat_merged['observation_period_start_date'] = pd.to_datetime(
    {
        'year': ddf_adm_pat_merged['year_of_admit'],
        'month': ddf_adm_pat_merged['month_of_admit'],
        'day': ddf_adm_pat_merged['day_of_admit']
    }
)

print(ddf_adm_pat_merged[['observation_period_start_date']])

       observation_period_start_date
0                         2012-03-20
1                         2017-05-25
2                         2020-05-22
3                         2013-09-03
4                         2014-01-15
...                              ...
431226                    2009-08-09
431227                    2009-07-05
431228                    2015-09-24
431229                    2015-10-27
431230                    2019-07-28

[431231 rows x 1 columns]


In [18]:
ddf_adm_pat_merged['discharged_year'] = ddf_adm_pat_merged['dischtime'].dt.year
ddf_adm_pat_merged['year_of_discharge'] = ddf_adm_pat_merged['discharged_year'] - ddf_adm_pat_merged['diff_year']
ddf_adm_pat_merged['month_of_discharge'] = ddf_adm_pat_merged['dischtime'].dt.month
ddf_adm_pat_merged['day_of_discharge'] = ddf_adm_pat_merged['dischtime'].dt.day
ddf_adm_pat_merged['day_of_discharge'] = ddf_adm_pat_merged.apply(
    lambda row: min(row['day_of_discharge'], pd.Period(year=row['year_of_discharge'], month=row['month_of_discharge'], freq='M').days_in_month),
    axis=1
)
ddf_adm_pat_merged['observation_period_end_date'] = pd.to_datetime(
    {
        'year': ddf_adm_pat_merged['year_of_discharge'],
        'month': ddf_adm_pat_merged['month_of_discharge'],
        'day': ddf_adm_pat_merged['day_of_discharge']
    }
)
print(ddf_adm_pat_merged[['observation_period_end_date']])

       observation_period_end_date
0                       2012-03-23
1                       2017-05-26
2                       2020-05-24
3                       2013-09-06
4                       2014-01-18
...                            ...
431226                  2009-08-10
431227                  2009-07-12
431228                  2015-09-30
431229                  2015-10-30
431230                  2019-07-31

[431231 rows x 1 columns]


In [19]:
ddf_adm_pat_merged['period_type_concept_id'] = 32828
print(ddf_adm_pat_merged['period_type_concept_id'])

0         32828
1         32828
2         32828
3         32828
4         32828
          ...  
431226    32828
431227    32828
431228    32828
431229    32828
431230    32828
Name: period_type_concept_id, Length: 431231, dtype: int64


In [20]:
ddf_adm_pat_merged_copy =  ddf_adm_pat_merged.copy()
ddf_adm_pat_merged_copy

,subject_id,hadm_id,admittime,dischtime,anchor_year,anchor_year_group,median_anchor_year,admittime_year,diff_year,year_of_admit,month_of_admit,day_of_admit,observation_period_start_date,discharged_year,year_of_discharge,month_of_discharge,day_of_discharge,observation_period_end_date,period_type_concept_id
0,10467237,20000019,2159-03-20 21:08:00,2159-03-23 16:54:00,2159,2011 - 2013,2012,2159,147,2012,3,20,2012-03-20,2159,2012,3,23,2012-03-23,32828
1,16925328,20000024,2151-05-25 21:44:00,2151-05-26 18:36:00,2143,2008 - 2010,2009,2151,134,2017,5,25,2017-05-25,2151,2017,5,26,2017-05-26,32828
2,19430048,20000034,2174-05-22 19:13:00,2174-05-24 17:27:00,2172,2017 - 2019,2018,2174,154,2020,5,22,2020-05-22,2174,2020,5,24,2020-05-24,32828
3,18910522,20000041,2143-09-03 07:15:00,2143-09-06 13:50:00,2142,2011 - 2013,2012,2143,130,2013,9,3,2013-09-03,2143,2013,9,6,2013-09-06,32828
4,11146739,20000057,2190-01-15 17:07:00,2190-01-18 16:05:00,2188,2011 - 2013,2012,2190,176,2014,1,15,2014-01-15,2190,2014,1,18,2014-01-18,32828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,10382924,29999723,2170-08-09 18:56:00,2170-08-10 18:30:00,2170,2008 - 2010,2009,2170,161,2009,8,9,2009-08-09,2170,2009,8,10,2009-08-10,32828
431227,11326722,29999745,2160-07-05 22:24:00,2160-07-12 12:50:00,2160,2008 - 2010,2009,2160,151,2009,7,5,2009-07-05,2160,2009,7,12,2009-07-12,32828
431228,12133002,29999809,2172-09-24 21:27:00,2172-09-30 17:04:00,2166,2008 - 2010,2009,2172,157,2015,9,24,2015-09-24,2172,2015,9,30,2015-09-30,32828
431229,16572164,29999828,2180-10-27 07:15:00,2180-10-30 17:25:00,2180,2014 - 2016,2015,2180,165,2015,10,27,2015-10-27,2180,2015,10,30,2015-10-30,32828


In [21]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')
omopcursor = conn.cursor()

In [23]:
observation_period_data = ddf_adm_pat_merged_copy[['hadm_id', 'subject_id', 'observation_period_start_date', 'observation_period_end_date', 'period_type_concept_id']]

for index, row in observation_period_data.iterrows():   
    sql = "INSERT INTO OBSERVATION_PERIOD (observation_period_id, person_id, observation_period_start_date, observation_period_end_date, period_type_concept_id) VALUES (%s, %s, %s, %s, %s)"
    
    omopcursor.execute(sql, (
        row['hadm_id'],
        row['subject_id'],
        row['observation_period_start_date'],
        row['observation_period_end_date'],
        row['period_type_concept_id']
    ))

conn.commit()
print("Insertion was successful.")

Insertion was successful.


In [24]:
null_or_nat_count = ddf_adm_pat_merged['admittime'].isna().sum()
print("Number of null or NaT values in the 'admittime' column:", null_or_nat_count)
null_or_nat_count = ddf_adm_pat_merged['dischtime'].isna().sum()
print("Number of null or NaT values in the 'dischtime' column:", null_or_nat_count)

Number of null or NaT values in the 'admittime' column: 0
Number of null or NaT values in the 'dischtime' column: 0


In [25]:
null_or_nat_count = ddf_adm_pat_merged['observation_period_start_date'].isna().sum()
print("Number of null or NaT values in the 'observation_period_start_date' column:", null_or_nat_count)
null_or_nat_count = ddf_adm_pat_merged['observation_period_end_date'].isna().sum()
print("Number of null or NaT values in the 'observation_period_end_date' column:", null_or_nat_count)

Number of null or NaT values in the 'observation_period_start_date' column: 0
Number of null or NaT values in the 'observation_period_end_date' column: 0
